In [32]:
import os
import pandas as pd
import zipfile
import io
import sqlite3


config = {
    "APE": {
        "code_column_number": 1,
        "stock_column_number": 2,
        "process_func": lambda x: 0 if x == "No" else (10 if x == "YES" else 0),
    },
    "BET": {
        "code_column_number": 1,
        "stock_column_number": 2,
        "process_func": lambda x: 10 if x > 10 else x,
    },
    "BGA": {
        "code_column_number": 1,
        "stock_column_number": 2,
        "process_func": lambda x: 10 if x > 10 else x,
    },
    "COM": {
        "code_column_number": 1,
        "stock_column_number": 3,
        "process_func": lambda x: 10 if x > 10 else x,
    },
    "FAI": {
        "code_column_number": 1,
        "stock_column_number": 2,
        "process_func": lambda x: 10 if x > 10 else x,
    },
    "FEB": {
        "code_column_number": 1,
        "stock_column_number": 3,
        "process_func": lambda x: 10 if x > 10 else x,
    },
    "FIR": {
        "code_column_number": 1,
        "stock_column_number": 2,
        "process_func": lambda x: 10 if x == "Y" else 0,
    },
    "FPS": {
        "code_column_number": 1,
        "stock_column_number": 4,
        "process_func": lambda x: 10 if x > 10 else x,
    },
    "JUR": {
        "code_column_number": 1,
        "stock_column_number": 2,
        "process_func": lambda x: 10 if x > 10 else x,
    },
    "KLA": {
        "code_column_number": 1,
        "stock_column_number": 2,
        "process_func": lambda x: x,
    },
    "KYB": {
        "code_column_number": 1,
        "stock_column_number": 2,
        "process_func": lambda x: 0 if x == "N" else (10 if x == "Y" else 0),
    },
    "MOT": {
        "code_column_number": 1,
        "stock_column_number": 3,
        "process_func": lambda x: 10 if x > 10 else x,
    },
    "ROL": {
        "code_column_number": 1,
        "stock_column_number": 3,
        "process_func": lambda x: 10 if x == "In Stock" else 0,
    },
    "RTG": {
        "code_column_number": 1,
        "stock_column_number": 2,
        "process_func": lambda x: 0 if str(x) == "B152381" else 20,
    },
    "SMP": {
        "code_column_number": 1,
        "stock_column_number": 2,
        "process_func": lambda x: 10 if x == 0 else 0,
    },
}


def upload_to_sqlite(df, table_name, if_exists, db_path="data.db"):
    conn = sqlite3.connect(db_path)
    df.to_sql(table_name, conn, if_exists=if_exists, index=False)


def read_from_sqlite(table_name, db_path="data.db"):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    conn.close()
    return df


def process_inventory_data(days=7):
    inventory_df = read_from_sqlite("inventory")

    print(inventory_df)

    # Convert timestamp to datetime
    inventory_df["timestamp"] = pd.to_datetime(inventory_df["timestamp"])

    # Filter for dates in the past week
    one_week_ago = pd.Timestamp.now() - pd.Timedelta(days=days)
    inventory_df = inventory_df[inventory_df["timestamp"] >= one_week_ago]

    # Sort the filtered data
    inventory_df.sort_values(by=["supplier", "code", "timestamp"], inplace=True)

    # Group by supplier and code
    grouped_df = inventory_df.groupby(["supplier", "code"])

    # Calculate the net delta
    net_delta_df = grouped_df.agg(
        start_stock=("stock_calculation", "first"),
        end_stock=("stock_calculation", "last"),
        start_date=("timestamp", "first"),
        end_date=("timestamp", "last"),
        custom_label=("custom_label", "first")  # Add this line to include custom_label
    ).reset_index()

    # Calculate the delta
    net_delta_df["change"] = net_delta_df["end_stock"] - net_delta_df["start_stock"]

    # Select relevant columns for the final table
    final_df = net_delta_df[
        [
            "supplier",
            "code",
            "custom_label",
            "start_date",
            "end_date",
            "start_stock",
            "end_stock",
            "change",
        ]
    ]

    # Uncomment the following line if you want to filter out rows with no change
    # final_df = final_df[final_df["change"] != 0]

    return final_df


def process_dataframe(config_key, file):
    df = pd.read_excel(file)
    code_column = df.iloc[:, config[config_key]["code_column_number"] - 1]
    stock_column = df.iloc[:, config[config_key]["stock_column_number"] - 1]
    df_output = pd.DataFrame(
        {
            "supplier": config_key,
            "code": code_column,
            "custom_label": code_column.apply(lambda code: f"UKD-{config_key}-{code}"),
            "stock": stock_column,
            "stock_calculation": stock_column.apply(config[config_key]["process_func"]),
            "timestamp": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"),
        }
    )
    return df_output


item_ids = pd.read_csv("item_ids.csv")
upload_to_sqlite(item_ids, "item_ids", "replace")

inventory_df = process_inventory_data()

upload_to_sqlite(inventory_df, "inventory_changes", "replace")


       supplier    code    custom_label                 stock  \
0           APE  ABR101  UKD-APE-ABR101                   YES   
1           APE  ABR102  UKD-APE-ABR102                   YES   
2           APE  ABR103  UKD-APE-ABR103                   YES   
3           APE  ABR104  UKD-APE-ABR104                   YES   
4           APE  ABR105  UKD-APE-ABR105                   YES   
...         ...     ...             ...                   ...   
379568      SMP    W654    UKD-SMP-W654  WSL GLOW PLUG          
379569      SMP    W658    UKD-SMP-W658  WSL GLOW PLUG          
379570      SMP    W660    UKD-SMP-W660  WSL GLOW PLUG          
379571      SMP    W661    UKD-SMP-W661  WSL GLOW PLUG          
379572      SMP  WS1044  UKD-SMP-WS1044  COOLANT TEMP SENSOR    

        stock_calculation            timestamp  
0                    10.0  2024-07-31 14:17:36  
1                    10.0  2024-07-31 14:17:36  
2                    10.0  2024-07-31 14:17:36  
3                    10

In [33]:
inventory_df

,supplier,code,custom_label,start_date,end_date,start_stock,end_stock,change
0,APE,ABR101,UKD-APE-ABR101,2024-07-31 14:17:36,2024-07-31 14:17:36,10.0,10.0,0.0
1,APE,ABR102,UKD-APE-ABR102,2024-07-31 14:17:36,2024-07-31 14:17:36,10.0,10.0,0.0
2,APE,ABR103,UKD-APE-ABR103,2024-07-31 14:17:36,2024-07-31 14:17:36,10.0,10.0,0.0
3,APE,ABR104,UKD-APE-ABR104,2024-07-31 14:17:36,2024-07-31 14:17:36,10.0,10.0,0.0
4,APE,ABR105,UKD-APE-ABR105,2024-07-31 14:17:36,2024-07-31 14:17:36,10.0,10.0,0.0
...,...,...,...,...,...,...,...,...
373062,SMP,WVL006,UKD-SMP-WVL006,2024-07-31 14:17:47,2024-07-31 14:17:47,0.0,0.0,0.0
373063,SMP,WVL008,UKD-SMP-WVL008,2024-07-31 14:17:47,2024-07-31 14:17:47,0.0,0.0,0.0
373064,SMP,WVL009,UKD-SMP-WVL009,2024-07-31 14:17:47,2024-07-31 14:17:47,0.0,0.0,0.0
373065,SMP,WVL010,UKD-SMP-WVL010,2024-07-31 14:17:47,2024-07-31 14:17:47,0.0,0.0,0.0


In [34]:
def create_ebay_dataframe(inventory_df, item_ids):
    # Create a new dataframe with renamed columns
    ebay_df = inventory_df.rename(
        columns={"custom_label": "CustomLabel", "end_stock": "Quantity"}
    )

    ebay_df["Action"] = "Revise"
    ebay_df["SiteID"] = "UK"
    ebay_df["Currency"] = "GBP"

    ebay_df = ebay_df[
        [
            "Action",
            "CustomLabel",
            "SiteID",
            "Currency",
            "Quantity",
        ]
    ]

    # Remove rows with null values in the Quantity column
    ebay_df = ebay_df.dropna(subset=["Quantity"])

    # Convert Quantity to integer type
    ebay_df["Quantity"] = ebay_df["Quantity"].astype(int)

    # Join ebay_df with item_ids on the "custom_label" column
    ebay_df = ebay_df.merge(
        item_ids, left_on="CustomLabel", right_on="custom_label", how="left"
    )
    ebay_df = ebay_df.drop(columns=["custom_label"])
    ebay_df = ebay_df.rename(columns={"item_id": "ItemID"})
    
    return ebay_df


ebay_df = create_ebay_dataframe(inventory_df, item_ids)

In [35]:
ebay_df

,Action,CustomLabel,SiteID,Currency,Quantity,ItemID
0,Revise,UKD-APE-ABR101,UK,GBP,10,1.448674e+11
1,Revise,UKD-APE-ABR101,UK,GBP,10,1.156496e+11
2,Revise,UKD-APE-ABR101,UK,GBP,10,1.448689e+11
3,Revise,UKD-APE-ABR101,UK,GBP,10,1.156504e+11
4,Revise,UKD-APE-ABR101,UK,GBP,10,1.448690e+11
...,...,...,...,...,...,...
650022,Revise,UKD-SMP-WVL006,UK,GBP,0,1.255806e+11
650023,Revise,UKD-SMP-WVL008,UK,GBP,0,1.155805e+11
650024,Revise,UKD-SMP-WVL009,UK,GBP,0,1.155805e+11
650025,Revise,UKD-SMP-WVL010,UK,GBP,0,1.255806e+11


In [36]:
inventory_df

,supplier,code,custom_label,start_date,end_date,start_stock,end_stock,change
0,APE,ABR101,UKD-APE-ABR101,2024-07-31 14:17:36,2024-07-31 14:17:36,10.0,10.0,0.0
1,APE,ABR102,UKD-APE-ABR102,2024-07-31 14:17:36,2024-07-31 14:17:36,10.0,10.0,0.0
2,APE,ABR103,UKD-APE-ABR103,2024-07-31 14:17:36,2024-07-31 14:17:36,10.0,10.0,0.0
3,APE,ABR104,UKD-APE-ABR104,2024-07-31 14:17:36,2024-07-31 14:17:36,10.0,10.0,0.0
4,APE,ABR105,UKD-APE-ABR105,2024-07-31 14:17:36,2024-07-31 14:17:36,10.0,10.0,0.0
...,...,...,...,...,...,...,...,...
373062,SMP,WVL006,UKD-SMP-WVL006,2024-07-31 14:17:47,2024-07-31 14:17:47,0.0,0.0,0.0
373063,SMP,WVL008,UKD-SMP-WVL008,2024-07-31 14:17:47,2024-07-31 14:17:47,0.0,0.0,0.0
373064,SMP,WVL009,UKD-SMP-WVL009,2024-07-31 14:17:47,2024-07-31 14:17:47,0.0,0.0,0.0
373065,SMP,WVL010,UKD-SMP-WVL010,2024-07-31 14:17:47,2024-07-31 14:17:47,0.0,0.0,0.0


In [37]:
ebay_df

,Action,CustomLabel,SiteID,Currency,Quantity,ItemID
0,Revise,UKD-APE-ABR101,UK,GBP,10,1.448674e+11
1,Revise,UKD-APE-ABR101,UK,GBP,10,1.156496e+11
2,Revise,UKD-APE-ABR101,UK,GBP,10,1.448689e+11
3,Revise,UKD-APE-ABR101,UK,GBP,10,1.156504e+11
4,Revise,UKD-APE-ABR101,UK,GBP,10,1.448690e+11
...,...,...,...,...,...,...
650022,Revise,UKD-SMP-WVL006,UK,GBP,0,1.255806e+11
650023,Revise,UKD-SMP-WVL008,UK,GBP,0,1.155805e+11
650024,Revise,UKD-SMP-WVL009,UK,GBP,0,1.155805e+11
650025,Revise,UKD-SMP-WVL010,UK,GBP,0,1.255806e+11
